# Find recipe using Apriori algorithm

In [8]:
import numpy as np
import pandas as pd

from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser

# Prepare Data

In [11]:
import pandas as pd
df = pd.read_csv('/train.csv')
df.head()

,label,achiote paste,achiote powder,acini di pepe,acorn squash,active dry yeast,adobo sauce,adobo seasoning,adzuki beans,agave nectar,...,yellow rock sugar,yellow squash,yellow summer squash,yellow tomato,yellowfin tuna,yogurt cheese,yucca root,yukon gold potatoes,yuzu juice,zucchini blossoms
0,Infineon Raceway Baked Beans,0,0,0,0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Sour Cream Noodle Bake,0,0,0,0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Middle-Eastern Eggplant Rounds,0,0,0,0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Saffron Jewel Rice,0,0,0,0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Thai Sweet and Sour Wings,0,0,0,0,0.0,0.0,0,0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
import pandas as pd

try:
    df = pd.read_csv('C:/Users/2021PITAI176/Downloads/train.csv')
except FileNotFoundError as e:
    print(f"Error: {e}")

Error: [Errno 2] No such file or directory: 'C:/Users/2021PITAI176/Downloads/train.csv'


In [ ]:
features = df.iloc[:, 1:]
labels = df.iloc[:, 0]

# Additional functionality

In [13]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))


def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, recipes, labels, itemSetWithSup, minConf):
    rules = []
    lbls = []

    sets_compare = [set(re) for re in recipes]

    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:

                ss = set([es for es in s])
                if(len(ss.intersection(sets_compare[0])) > 0 and len(ss.intersection(sets_compare[1])) > 0):
                    lbls.append([labels[0], labels[1]])
                if(len(ss.intersection(sets_compare[0])) > 0):
                    lbls.append(labels[0])
                if(len(ss.intersection(sets_compare[1])) > 0):
                    lbls.append(labels[1])

                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules, lbls


def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))

    return tempItemSet

# Apriori pseudocode

<pre>
L[1] = {frequent 1-itemsets};
for (k=2; L[k-1] != 0; k ++) do begin
    // perform self-joining
    C[k] = getUnion(L[k-1])
    // remove pruned supersets
    C[k] = pruning(C[k])
    // get itemsets that satisfy minSup
    L[k] = getAboveMinSup(C[k], minSup)
end
Answer = Lk (union)
</pre>

# Apriori implementation

In [14]:
def apriori(itemSetList, labels, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules, lbls = associationRule(globalFreqItemSet, itemSetList, labels, globalItemSetWithSup, minConf)
    #rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules, lbls

# Compare two receipes

In [17]:
idx_to_column = {k:v for k,v in enumerate(features.columns.values)}
column_to_idx = {v:k for k,v in enumerate(features.columns.values)}

In [16]:
features = pd.DataFrame({'column1': [1, 2, 3], 'column2': [4, 5, 6]})

In [18]:
def get_random_recipes(features, labels, nr):
    ids = np.random.randint(0, len(features), size=nr).tolist()
    idxs = np.array([np.arange(len(features.columns.values)).tolist() for x in range(nr)])
    multiply = features.iloc[ids].values.astype(np.int32) * idxs
    idx_arr  = [np.where(mult != 0)[0].tolist() for mult in multiply]
    recipes = []
    for arr_idx in idx_arr:
        temp_result = []
        for idx in arr_idx:
            temp_result.append(idx_to_column[idx])
        recipes.append(temp_result)
    return recipes, labels[ids].values

In [19]:
def print_recipe(recipe, label):
    print("-" * 50)
    print("Recipe: ", label)
    print("-" * 50)
    for item in recipe:
        print(item)

In [27]:
# Print the features DataFrame
print(features.head())

# Print the labels DataFrame
print(labels.head())

   column1  column2
0        1        4
1        2        5
2        3        6
    cuisine
0   italian
1   mexican
2      thai
3  american
4   chinese


In [28]:
def get_random_recipes(features, labels, nr):
    # Get random indices from the features DataFrame
    ids = features.sample(n=nr).index

    # Select the corresponding rows from the labels DataFrame
    labels = labels.loc[ids]

    # Create a list of recipes and labels
    recipes = []
    lbls = []
    for idx in ids:
        recipes.append(features.loc[idx].values)
        lbls.append(labels.loc[idx].values)

    return recipes, lbls

In [34]:
# Add a new column to the labels DataFrame with the index 2
labels.loc[2] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Get random recipes and labels
recipes, lbls = get_random_recipes(features, labels, nr=2)

ValueError: could not broadcast input array from shape (30,) into shape (1,)

In [31]:
print(labels.shape)

(10, 1)


In [32]:
# Add a new column to the labels DataFrame with the index 2
labels.loc[2] = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

# Get random recipes and labels
recipes, lbls = get_random_recipes(features, labels, nr=2)

In [33]:
print(labels)

                                             cuisine
0                                            italian
1                                            mexican
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3                                           american
4                                            chinese
5                                             french
6                                             indian
7                                              greek
8                                            spanish
9                                         vietnamese


In [35]:
freqItemSet, rules, lbls = apriori(recipes, lbls, minSup=0.5, minConf=0.5)

In [36]:
def find_recipe(ingredients, rules, lbls):
    rules_dic = {}
    for i, item in enumerate(np.array(rules)[:, :-1]):
        item = [list(i) for i in item]
        rules_dic[str(item)] = lbls[i]

    result = None

    try:
        result = rules_dic[str(ingredients)]
    except:
        result = "Recipe not found"

    return result

In [37]:
print('Try to find recipe using ingredients: {0} and {1}'.format(recipes[0][0], recipes[0][1]))
found_recipe = find_recipe([[recipes[0][0]], [recipes[0][1]]], rules, lbls)
print('Found recipe: ', found_recipe)

Try to find recipe using ingredients: 2 and 5
Found recipe:  ['mexican']


In [38]:
ingredients_to_try = [['vanilla vodka'], ['coconut extract']]
print('Testing not finding ingredients')
print('Using the following ingredients: {0} and {1}'.format(ingredients_to_try[0][0], ingredients_to_try[1][0]))
print(find_recipe(ingredients_to_try, rules, lbls))

Testing not finding ingredients
Using the following ingredients: vanilla vodka and coconut extract
Recipe not found


# Conclusion

<pre>
Apriori algorithm is really useful for data minning.
We can use it easily to build an application that given ingredients, gives you the recipe.
</pre>